In [3]:
using Pkg
Pkg.activate("C:\\Users\alili\\.julia\\environments\\v1.5\\Project.toml") # change path 
Pkg.instantiate()

 Activating new environment at `C:\Userslili\.julia\environments\v1.5\Project.toml`
   Updating registry at `C:\Users\alili\.julia\registries\General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


LoadError: IOError: mkdir: invalid argument (EINVAL)

In [2]:
using JuMP, LinearAlgebra, Plots, StatsPlots, CPLEX, Statistics, CSV, DataFrames

In [20]:
names(scenarios_prices)[3:length(names(scenarios_prices))]

7-element Array{String,1}:
 "22"
 "23"
 "24"
 "1"
 "2"
 "3A"
 "4"

In [66]:
#read scenario data
scenarios_hexagons = CSV.read("scenarios_multinomial.csv", DataFrame)
scenarios_prices = CSV.read("priceScenarions3.csv", DataFrame)

SOC = 0.33
hexagons = names(scenarios_hexagons)[1:length(names(scenarios_hexagons))-1] # vector of hexagons (strings)
hours = names(scenarios_prices)[3:length(names(scenarios_prices))] #vector of hours for charging (strings)
num_hours = length(hours) # duration for charging
car_kWh = 30 # capacity of each car

#demand mxn (m=hexagon, n=scenario) in kWh summe über car*max_kWh*(1-SOC)
demand_cars = transpose(convert(Matrix,scenarios_hexagons[:,hexagons]))
demand_kWh = demand_cars.*car_kWh.*(1-SOC)

#prices mxn (m=hours, n=scenario)
prices = transpose(convert(Matrix,scenarios_prices[:,hours]))

probability_hexagons = transpose(scenarios_hexagons.probs) # probabilities for each scenario (#cars per hexagon)
probability_prices = transpose(scenarios_prices.Probability) # probabilities for each scenario (timeseries prices)

P = [3, 7, 11] # different charger types in kW
charger_cost = [1000, 800, 500] # investment cost for charger 3 kW, 7 kW und 11 kW in €/kW
opportunity_cost = 5000 # cost of demand not served


5000

In [67]:
function charger_siting(SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, prices, 
        probability_hexagons, probability_prices, P, charger_cost, opportunity_cost)
    
    # Define Sets
    Γ = Array{Int}(1:size(hexagons,1)) # Set with hexagons
    Σ = Array{Int}(1:size(P,1)) # Set with charger types 
    Ω = Array{Int}(1:size(probability_hexagons,1)) # Set with scenarios for hexagons
    Τ = Array{Int}(1:size(hours,1)) # Set with scenarios for prices
    Ρ = Array{Int}(1:size(probability_prices,1)) # Set with scenarios for prices

    # Initialize model    
    m = Model(CPLEX.Optimizer)

    # Define variables
    @variable(m, 0 <= x_charger[i=Γ, j=Σ], Int) # number of specific charger type in each hexagon
    @variable(m, 0 <= E_supplied[i=Γ, j=Σ, k=Ω, t=Τ]) # supplied energy in each hexagon kWh
    @variable(m, 0 <= E_not_supplied[i=Γ, k=Ω]) # demand not covered kWh
    @variable(m, 0 <= cars_supplied[i=Γ, j=Σ, k=Ω], Int)
    @variable(m, 0 <= cars_not_supplied[i=Γ, j=Σ, k=Ω], Int)

    # Objective function 
    @expression(m, invest_cost[i=Γ, j=Σ], charger_cost[j].*x_charger[i,j].*P[j]) # first stage investment cost
    @expression(m, charging_opportunity_cost[i=Γ, k=Ω], opportunity_cost.*E_not_supplied[i,k].*probability_hexagons[k])
    @expression(m, charging_cost[i=Γ, j=Σ, k=Ω, t=Τ, p=Ρ], prices[t,p].*E_supplied[i,j,k,t]
                                                         .*probability_hexagons[k]
                                                         .*probability_prices[p]) #minimize cost of charging
    
    @objective(m, Min, sum(invest_cost) 
                     + sum(charging_opportunity_cost)
                     + sum(charging_cost)
                      )

    # Constraints
    # Eq. Balance equation: Demand in each hexagon has to equal demand. Loss of load is possible
    @constraint(m, eq_balance[i=Γ,k=Ω], sum(sum(E_supplied[i,j,k,t] for j in Σ) for t in Τ) + E_not_supplied[i,k] == demand_kWh[i,k])

    # Eq. energy limit: rated_power*chargingtime must not exeed energy
    @constraint(m, eq_cap[i=Γ,j=Σ,k=Ω,t=T], E_supplied[i,j,k,t] <= x_charger[i,j].*P[j])

    # Eq. supplied cars and not supplied cars have to equal amount of cars in each hexagon
    @constraint(m, eq_cars_supplied[i=Γ,j=Σ,k=Ω], sum(cars_supplied[i,j,k] for j in Σ) 
                                                     + sum(cars_not_supplied[i,j,k] for j in Σ)
                                                       == demand_cars[i,k])

    # Eq. maximum one car per charging station
    @constraint(m, eq_max_cars[i=Γ,j=Σ,k=Ω], cars_supplied[i,j,k] <= x_charger[i,j])

    # Eq. 
    @constraint(m, eq_max_supply[i=Γ,j=Σ,k=Ω], sum(E_supplied[i,j,k,t] for t in Τ) <= cars_supplied[i,j,k].*car_kWh.*(1-SOC))


    # Run optimization
    optimize!(m)

    # Evaluate resuluts
    obj = objective_value(m)
    charger_in_hex = value.(x_charger)
    
    return charger_in_hex, x_charger, E_not_supplied, E_supplied, cars_not_supplied, cars_supplied
end


charger_siting (generic function with 2 methods)

In [68]:
charger_in_hex, x_charger, E_not_supplied, E_supplied, cars_not_supplied, cars_supplied = charger_siting(
    SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, prices, 
        probability_hexagons, probability_prices, P, charger_cost, opportunity_cost)

println(charger_in_hex)

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
Found incumbent of value 1.9111191e+07 after 0.00 sec. (0.21 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 535 rows and 2281 columns.
MIP Presolve modified 48 coefficients.
Reduced MIP has 726 rows, 726 columns, and 1716 nonzeros.
Reduced MIP has 77 binaries, 385 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (2.76 ticks)
Probing time = 0.00 sec. (0.06 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 715 rows and 715 columns.
Reduced MIP has 11 rows, 11 columns, and 26 nonzeros.
Reduced MIP has 7 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.39 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 11 rows, 11 columns, and 26 nonzeros.
Reduced MIP has 7 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Clique table members: 4.
MIP emphasis: 